# Operaciones con Tokens: Descomposición de Palabras y Búsqueda Vectorial Eficiente

# Disección de Tokens en GPT-2: El Caso de la "Banana Cósmica"

En esta práctica vamos a:
1.  Ver cómo GPT-2 rompe palabras comunes (`banana`, `cosmic`) en sub-unidades.
2.  Analizar la similitud entre estas partes.
3.  Optimizar la búsqueda de similitud semántica pasando de un bucle `for` lento a álgebra lineal vectorizada (Matrix Multiplication).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

from transformers import GPT2Model, GPT2Tokenizer

# Modelo GPT-2 preentrenado y tokenizador
gpt2 = GPT2Model.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Obtener la matriz de Word Token Embeddings
embeddings = gpt2.wte.weight.detach().numpy()
print(f"Matriz de embeddings cargada: {embeddings.shape}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Matriz de embeddings cargada: (50257, 768)


In [2]:
## Ejercicio 1: La Banana Cósmica (Descomposición)

Vamos a analizar `banana`, `apple` y `cosmic`.
* **Apple:** Probablemente sea 1 solo token.
* **Banana:** GPT-2 suele dividirla en `ban` + `ana`.
* **Cosmic:** Suele ser `cos` + `mic`.

El código a continuación visualiza los vectores de estos fragmentos y luego compara la similitud entre el prefijo (`cos`) y diferentes sufijos (`mic` vs `ine` de "cosine").

SyntaxError: invalid syntax (ipython-input-3951285546.py, line 3)

In [ ]:
# Palabras de interés
word1 = 'banana'
word2 = 'apple'
word3 = 'cosmic'

# Descomposición y análisis de tokens
for w in [word1, word2, word3]:
  t = tokenizer.encode(w)
  # Decodificamos cada sub-token para ver cómo rompió la palabra
  tokens_decoded = [tokenizer.decode(ti) for ti in t]
  print(f'"{w}" comprende {len(t)} tokens:\n   {tokens_decoded}\n')

# Configurar la geometría de la figura
fig = plt.figure(figsize=(10, 7))
gs = GridSpec(2, 2)
ax0 = fig.add_subplot(gs[0, :])
ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[1, 1])

# Plot de los embeddings por dimensión
colors = 'krb'
linestyles = ['-', '--', '-.', ':']

for idx, word in enumerate([word1, word2, word3]):
  wordidx = tokenizer.encode(word)
  # Iterar sobre cada sub-token de la palabra
  for j in range(len(wordidx)):
    label = f"{word} (part {j+1})"
    ax0.plot(embeddings[wordidx[j]], color=colors[idx], linestyle=linestyles[j], label=label)

ax0.set(xlabel='Dimensión', title='Embeddings (Sub-tokens)', xlim=[-1, embeddings.shape[1]+1])
ax0.legend()

# --- Similitud Coseno entre 'ban' y 'ana' ---
# Nota: 'ana' como sufijo suele llevar espacio en GPT2, pero probaremos el token directo
# Usamos [0] para sacar el entero de la lista
idx_ban = tokenizer.encode('ban')[0]
idx_ana = tokenizer.encode('ana')[0]

v1 = embeddings[idx_ban]
v2 = embeddings[idx_ana]

# Cálculo manual del coseno
cossim = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# Plot de correlación
ax1.plot(v1, v2, 'ko', markerfacecolor=[.7, .7, .7, .6])
ax1.set(xlim=[-.4, .4], ylim=[-.4, .4], xlabel='"ban" embedding', ylabel='"ana" embedding',
        title=f'Similitud Coseno = {cossim:.3f}')


# --- Plot del "cos" para dos terminaciones (mic vs ine) ---
# Comparamos "cos" de cosmic con "mic" y con "ine" (de cosine)
idx_cos = tokenizer.encode('cos')[0]
v_cos = embeddings[idx_cos]

for s in ['mic', 'ine']:
  idx_s = tokenizer.encode(s)[0]
  v_suffix = embeddings[idx_s]

  # Similitud coseno
  cossim = np.dot(v_cos, v_suffix) / (np.linalg.norm(v_cos) * np.linalg.norm(v_suffix))

  # Plot
  ax2.plot(v_cos, v_suffix, 'o', label=f'cos-{s} (r={cossim:.2f})', alpha=0.6)

ax2.legend(fontsize=10)
ax2.set(xlim=[-.4, .4], ylim=[-.4, .4], xlabel='"cos" embedding', ylabel='"mic" o "ine" embedding')

# Toques finales
plt.tight_layout()
plt.show()

## Ejercicio 2: Búsqueda de Similitud (Método Lento)

Queremos encontrar qué tokens se parecen más a la palabra **"Mike"**.
Para ilustrar la complejidad computacional, primero lo haremos "a la fuerza bruta": un bucle `for` que recorre los 50,257 tokens uno por uno calculando la similitud.

> *Nota: Esto demuestra por qué necesitamos álgebra lineal. ¡Python puro es lento para esto!*

In [ ]:
# Obtener índice semilla y vector
seed_word = 'Mike'
seed = tokenizer.encode(seed_word)[0]
seedvect = embeddings[seed, :]

# Norma del vector semilla para el cálculo del coseno
seedvectNorm = np.linalg.norm(seedvect)

# Calcular similitud coseno con TODOS los otros tokens en un bucle
cossims = np.zeros(embeddings.shape[0])

print(f"Iniciando búsqueda lenta para '{seed_word}'...")
for idx, v in enumerate(embeddings):
  # Fórmula: (A . B) / (|A| * |B|)
  vNorm = np.linalg.norm(v)
  cossims[idx] = np.dot(seedvect, v) / (seedvectNorm * vNorm)

# Visualizar
plt.figure(figsize=(10, 4))
plt.plot(cossims, 'k.', alpha=.6, markerfacecolor='w', markersize=1)
plt.gca().set(xlim=[-5, len(cossims)+5], xlabel='Token index', ylabel='Similitud Coseno',
              title=f'Similitud Coseno con "{seed_word}"')
plt.show()

# Top 20 tokens más similares
top20 = np.argsort(cossims)[-20:]

print(f"\n--- Tokens más parecidos a '{seed_word}' ---")
for i in top20[::-1]: # Invertir para mostrar el más alto primero
    token_str = tokenizer.decode([i])
    print(f'"{token_str}" con similitud {cossims[i]:.4f}')

## Ejercicio 3: Optimización Vectorizada (Álgebra Lineal)

En lugar de hacer 50,000 iteraciones en Python, usamos el poder de las matrices.
1.  Normalizamos **toda** la matriz de embeddings de una sola vez.
2.  Hacemos un solo producto punto (Matrix-Vector multiplication).

Esto reduce el tiempo de cálculo de segundos a milisegundos.

In [ ]:
# 1. Normalizar la matriz de embeddings completa (en una nueva variable)
# Dividimos cada vector por su norma L2
# axis=1 asegura que calculamos la norma de cada fila (cada palabra)
vector_norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
Enorm = embeddings / vector_norms

# 2. Calcular matriz de similitudes (Vector semilla normalizado X Matriz Normalizada Transpuesta)
# Primero normalizamos el vector semilla
seedvectNormUnit = seedvect / seedvectNorm

# Producto punto masivo: (1 x 768) dot (768 x 50257) = (1 x 50257)
cossims2 = np.dot(Enorm, seedvectNormUnit)

print(f'Tamaño versión for-loop:   {cossims.shape}')
print(f'Tamaño versión vectorizada: {cossims2.shape}')

# Eliminar dimensiones extra si las hay (aunque np.dot suele devolver shape correcto aquí)
cossims2 = cossims2.squeeze()

# Inspeccionar de nuevo
print('\n** Después de squeeze:**')
print(f'Tamaño versión for-loop:   {cossims.shape}')
print(f'Tamaño versión vectorizada: {cossims2.shape}')

# --- Comparación Visual ---
plt.figure(figsize=(10, 4))
# Graficamos una muestra (cada 100 puntos) para no saturar el gráfico
plt.plot(range(0, len(cossims), 100), cossims[::100], 'o', label='Bucle For', markersize=8, alpha=0.5)
plt.plot(range(0, len(cossims), 100), cossims2[::100], '+', label='Vectorizado', markersize=8, color='r')

plt.legend()
plt.gca().set(xlabel='Token idx (muestreo cada 100)', ylabel='Similitud Coseno',
              title=f'Correlación entre métodos = {np.corrcoef(cossims, cossims2)[0,1]:.5f}')
plt.show()

# Análisis de Resultados: Granularidad de Tokens y Eficiencia Vectorial

# Conclusiones del Experimento

### 1. El Fenómeno Sub-word ("Banana")
Al analizar `banana` y `cosmic`, confirmamos que GPT-2 no tiene una entrada única para estas palabras.
* **Banana** $\rightarrow$ `ban` + `ana`.
* **Cosmic** $\rightarrow$ `cos` + `mic`.
Esto significa que el modelo construye el significado de "banana" combinando dinámicamente estos dos vectores en sus capas internas. Los embeddings iniciales (`wte`) de `ban` y `ana` pueden no estar muy relacionados semánticamente entre sí (como vimos en el gráfico de dispersión, su similitud no es necesariamente alta), pero su **combinación secuencial** es lo que activa el concepto de fruta.

### 2. Vecinos Semánticos de "Mike"
Al buscar similitudes para "Mike", el modelo encontró nombres propios relacionados:
* `Dave`, `Steve`, `Chris`, `Dan`, `Matt`.
Esto confirma que el espacio vectorial agrupa **categorías semánticas**. Todos estos son nombres masculinos comunes en inglés, por lo que sus vectores viven en el mismo "vecindario" geométrico.

### 3. La Importancia de la Vectorización
El Ejercicio 3 es una lección fundamental de ingeniería de ML.
* El bucle `for` tardó visiblemente más (dependiendo de tu CPU).
* La versión vectorizada `np.dot` fue instantánea.
* El gráfico final confirma que los resultados son idénticos (Correlación = 1.000), pero la versión vectorizada es miles de veces más eficiente. Así es como funcionan los motores de búsqueda y la atención en los Transformers: nunca iteran, siempre multiplican matrices.